In [ ]:
import scipy.stats
class VotingScheme:
    def __init__(self, preferences, voting_vector):
        self.preferences = preferences
        self.voting_vector = voting_vector # must be the length of possible outcomes

    def execute_voting(self, preferences):
        outcome_scores = {}

        # Initialize outcomes with zero scores
        for voter_prefs in preferences.values():
            for outcome in voter_prefs:
                outcome_scores[outcome] = 0

        # Calculate scores based on preferences and voting vector
        for voter, prefs in preferences.items():
            for i, outcome in enumerate(prefs):
                outcome_scores[outcome] += self.voting_vector[i]

        # Sort outcomes by score and then alphabetically to break ties
        outcomes = sorted(outcome_scores.items(), key=lambda x: (-x[1], x[0]))
        outcome = [outcome for outcome, score in outcomes]

        return outcome, outcome_scores


    def calc_overall_happiness(self, outcome):
        happiness_score = 0.0
        num_voters = len(self.preferences)

        for voter, prefs in self.preferences.items():
            happiness_score += self.calc_ind_happiness(voter,outcome)

        return happiness_score / num_voters

    def calc_ind_happiness(self, voter, outcome_ranking):

      spearman_correlation, _ = scipy.stats.spearmanr(self.preferences[voter], outcome_ranking)
      return spearman_correlation

In [16]:
preferences = {'A': ['o_1', 'o_3', 'o_2','o_4'], 'B': ['o_2', 'o_1', 'o_3','o_4'], 'C': ['o_2', 'o_3', 'o_1','o_4'], 'D': ['o_1','o_2','o_3','o_4'], 'E': ['o_4','o_3','o_2','o_1'],'F': ['o_4','o_3','o_2','o_1'],'G': ['o_3','o_2','o_1','o_4']}

# Priority Voting
priority_scheme = VotingScheme(preferences, [1, 1, 0, 0])
outcome, results = priority_scheme.execute_voting(preferences)
print(outcome)
print(results)
print(priority_scheme.calc_overall_happiness(outcome))
# Borda Count (Assuming 4 outcomes)
borda_scheme = VotingScheme(preferences, [3, 2, 1, 0])
outcome, results = borda_scheme.execute_voting(preferences)
print(outcome)
print(results)
print(borda_scheme.calc_overall_happiness(outcome))


['o_3', 'o_2', 'o_1', 'o_4']
{'o_1': 3, 'o_3': 5, 'o_2': 4, 'o_4': 2}
0.34285714285714286
['o_2', 'o_3', 'o_1', 'o_4']
{'o_1': 10, 'o_3': 13, 'o_2': 13, 'o_4': 6}
0.34285714285714286


In [26]:
import itertools
import pandas as pd
import random
import networkx as nx
import sklearn
from sklearn.cluster import OPTICS

class MultiAgentTVAModel:
    """
    Represents a multi-agent Tactical Voting Analyst model.
    """

    def __init__(self, preferences, voting_scheme):
        """
        Initializes the model.

        Args:
            preferences: A dictionary representing the true preferences of each agent.
                         Format example: { agent_id: [preference_1, preference_2, ...] }
            voting_scheme:  An object representing the voting scheme in use.
        """

        self.preferences = preferences
        self.voting_scheme = voting_scheme
        self.agents = list(preferences.keys())  # Extract agent IDs

        # Initialize results (computed later)
        self.outcome = None
        self.overall_happiness = None
        self.strategic_voting_options = {}

    def calculate_initial_outcome(self):
        """
        Calculates the initial outcome and happiness levels based on true preferences.
        """

        self.outcome, scores = self.voting_scheme.execute_voting(self.preferences)
        self.overall_happiness = self.voting_scheme.calc_overall_happiness(self.outcome)

    def analyze_strategic_voting(self):
        """
        Analyzes strategic voting possibilities for each agent.

        * Simulates different potential voting strategies.
        * Explores collusion potential, where applicable.
        * Stores potential strategic options.
        """

        for agent_id in self.agents:
            # Analyze individual strategic options
            self.strategic_voting_options[agent_id] = self._analyze_individual_strategy(agent_id)

    def _analyze_individual_strategy(self, agent_id):
      """
      Analyzes individual strategic voting options for a single agent, optimizing for sparse voting vectors.

      Args:
        agent_id: The ID of the agent for which to analyze strategies.

      Returns:
        A list of dictionaries, each representing a potential strategic vote, containing:
            * 'strategic_vote': The manipulated preference list
            * 'happiness_gain': Improvement in happiness compared to truthful voting
      """

      true_prefs = self.preferences[agent_id]
      strategic_options = []

      # Find relevant positions (indices with non-zero weights)
      relevant_positions = [i for i, value in enumerate(self.voting_scheme.voting_vector) if value != 0]

      # Generate permutations only for relevant elements
      for relevant_permutation in itertools.permutations(true_prefs, len(relevant_positions)):
        strategic_vote = true_prefs.copy()

        # Insert elements from the permutation into relevant positions
        for i, outcome in zip(relevant_positions, relevant_permutation):
            strategic_vote[i] = outcome

        # Simulate outcome with strategic vote
        simulated_preferences = self.preferences.copy()
        simulated_preferences[agent_id] = list(strategic_vote)

        simulated_outcome, scores = self.voting_scheme.execute_voting(simulated_preferences)
        simulated_happiness = self.voting_scheme.calc_ind_happiness(agent_id, simulated_outcome)

        # Calculate happiness gain (if any)
        true_happiness = self.voting_scheme.calc_ind_happiness(agent_id, self.outcome)
        happiness_gain = simulated_happiness - true_happiness

        if happiness_gain > 0:  # Strategy is beneficial
            strategic_options.append({
                'strategic_vote': strategic_vote,
                'happiness_gain': happiness_gain
            })

      return strategic_options

    def _analyze_joint_strategic_votes(self, coalition):
        relevant_positions = [i for i, value in enumerate(self.voting_scheme.voting_vector) if value != 0]
        coalition_prefs = [self.preferences[agent_id] for agent_id in coalition]

        # Generate all joint permutations for the relevant positions
        # Generate constrained joint permutations
        all_joint_permutations = itertools.product(
            *[
                [
                    perm for perm in itertools.permutations(prefs, len(relevant_positions))
                    if perm != tuple(prefs[i] for i in relevant_positions)  # Filter
                ]
                for prefs in coalition_prefs
            ]
        )
        collusion_data = []  # Store potential collusions
        for joint_strategic_votes in all_joint_permutations:
            simulated_prefs = self.preferences.copy()
            for agent_id, strategic_vote in zip(coalition, joint_strategic_votes):
                simulated_prefs[agent_id] = list(strategic_vote)

            simulated_outcome, _ = self.voting_scheme.execute_voting(simulated_prefs)
            happiness_increases = []

            all_gains_positive = True
            for agent_id, strategic_vote in zip(coalition, joint_strategic_votes):
                simulated_happiness = self.voting_scheme.calc_ind_happiness(agent_id, simulated_outcome)
                true_happiness = self.voting_scheme.calc_ind_happiness(agent_id, self.outcome)
                happiness_increase = simulated_happiness - true_happiness
                happiness_increases.append(happiness_increase)

                if happiness_increase <= 0:
                    all_gains_positive = False
                    break

            if all_gains_positive:
                overall_happiness_increase = sum(happiness_increases) / len(coalition)
                collusion_data.append({
                    'coalition': coalition,
                    'strategic_votes': joint_strategic_votes,
                    'overall_happiness_increase': overall_happiness_increase,
                    'individual_happiness_increases': happiness_increases
                })

        # Create and return the DataFrame
        return pd.DataFrame(collusion_data)

    def create_collusion_graph(self):
      """
      Creates a graph representing potential collusions between agents.
      Returns:
        networkx.Graph: A graph where nodes are agents, and edges represent potential collusions.
        """

      G = nx.Graph()
      G.add_nodes_from(self.agents)

      # Analyze collusion potential for all pairs of agents
      for agent1, agent2 in itertools.combinations(self.agents, 2):
        coalition = (agent1, agent2)
        collusion_df = self._analyze_joint_strategic_votes(coalition)

        # If potential collusions are found
        if not collusion_df.empty:
            max_happiness_increase = collusion_df['overall_happiness_increase'].max()

            # Add an edge with weight representing potential benefit
            G.add_edge(agent1, agent2, weight=max_happiness_increase)

      return G

    import networkx as nx

    def find_and_analyze_cliques(self):
      """
      Finds cliques in the collusion graph and analyzes their potential impact.
      """

      G = self.create_collusion_graph()

      # Find cliques within the graph
      cliques = [clique for clique in nx.find_cliques(G) if len(clique) > 1]
      # Analyze each clique
      for clique in cliques:
        print("Analyzing clique:", clique)
        for agent in clique:
          print("agent: ",agent, " preference: ",preferences[agent])
        coalition_df = self._analyze_joint_strategic_votes(clique)

        # Display collusion_df containing details about the collusion
        print(coalition_df)



    def calculate_risk(self):
        """
        Calculates a risk metric for strategic voting, taking into account
        individual and collusive strategies.
        """
        # TODO: Replace placeholder
        return random.random()  # Placeholder risk value

In [27]:
# Set up the MultiAgentTVAModel
priority_model = MultiAgentTVAModel(preferences, priority_scheme)

# Initial outcome and happiness (could be pre-calculated if you wish)
priority_model.calculate_initial_outcome()

# Analyze strategic voting
priority_model.analyze_strategic_voting()

# Example Output (Focusing on agents 'A' and 'B')
print("STRATEGIC VOTING ANALYSIS (Plurality Count)")
print("---------------------------------------")

for agent_id, strategies in priority_model.strategic_voting_options.items():
    if agent_id in priority_model.agents:
        print(f"\nAgent: {agent_id}")
        print(f"True Preferences: {priority_model.preferences[agent_id]}")

        if strategies:
            print("Potential Strategic Votes:")
            for strategy in strategies:
                print(f"  - Strategic Vote: {strategy['strategic_vote']}")
                print(f"    Happiness Gain: {strategy['happiness_gain']}")
        else:
            print("No beneficial strategic votes found.")

STRATEGIC VOTING ANALYSIS (Plurality Count)
---------------------------------------

Agent: A
True Preferences: ['o_1', 'o_3', 'o_2', 'o_4']
Potential Strategic Votes:
  - Strategic Vote: ['o_1', 'o_2', 'o_2', 'o_4']
    Happiness Gain: 0.39999999999999997
  - Strategic Vote: ['o_1', 'o_4', 'o_2', 'o_4']
    Happiness Gain: 0.39999999999999997
  - Strategic Vote: ['o_3', 'o_2', 'o_2', 'o_4']
    Happiness Gain: 0.39999999999999997
  - Strategic Vote: ['o_2', 'o_1', 'o_2', 'o_4']
    Happiness Gain: 0.39999999999999997
  - Strategic Vote: ['o_2', 'o_3', 'o_2', 'o_4']
    Happiness Gain: 0.39999999999999997
  - Strategic Vote: ['o_4', 'o_1', 'o_2', 'o_4']
    Happiness Gain: 0.39999999999999997

Agent: B
True Preferences: ['o_2', 'o_1', 'o_3', 'o_4']
Potential Strategic Votes:
  - Strategic Vote: ['o_1', 'o_3', 'o_3', 'o_4']
    Happiness Gain: 0.39999999999999997
  - Strategic Vote: ['o_1', 'o_4', 'o_3', 'o_4']
    Happiness Gain: 0.39999999999999997
  - Strategic Vote: ['o_3', 'o_1', '

In [28]:
df = priority_model.find_and_analyze_cliques()

Analyzing clique: ['D', 'B']
agent:  D  preference:  ['o_1', 'o_2', 'o_3', 'o_4']
agent:  B  preference:  ['o_2', 'o_1', 'o_3', 'o_4']
   coalition           strategic_votes  overall_happiness_increase  \
0     [D, B]  ((o_1, o_3), (o_1, o_2))                         0.3   
1     [D, B]  ((o_1, o_3), (o_1, o_3))                         0.3   
2     [D, B]  ((o_1, o_3), (o_3, o_1))                         0.3   
3     [D, B]  ((o_1, o_4), (o_1, o_2))                         0.3   
4     [D, B]  ((o_2, o_1), (o_1, o_3))                         0.3   
5     [D, B]  ((o_2, o_1), (o_1, o_4))                         0.3   
6     [D, B]  ((o_2, o_1), (o_3, o_1))                         0.3   
7     [D, B]  ((o_2, o_1), (o_4, o_1))                         0.3   
8     [D, B]  ((o_3, o_1), (o_1, o_2))                         0.3   
9     [D, B]  ((o_3, o_1), (o_1, o_3))                         0.3   
10    [D, B]  ((o_3, o_1), (o_3, o_1))                         0.3   
11    [D, B]  ((o_4, o_1)